In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
file_path = "/content/drive/My Drive/traffic data/Ped_Crashes.csv"
collisions = pd.read_csv(file_path)

In [3]:
print("📋 Columns in 'collisions':")
print(collisions.columns.tolist())


📋 Columns in 'collisions':
['CASE_ID', 'ACCIDENT_YEAR', 'PROC_DATE', 'JURIS', 'COLLISION_DATE', 'COLLISION_TIME', 'OFFICER_ID', 'REPORTING_DISTRICT', 'DAY_OF_WEEK', 'CHP_SHIFT', 'POPULATION', 'CNTY_CITY_LOC', 'SPECIAL_COND', 'BEAT_TYPE', 'CHP_BEAT_TYPE', 'CITY_DIVISION_LAPD', 'CHP_BEAT_CLASS', 'BEAT_NUMBER', 'PRIMARY_RD', 'SECONDARY_RD', 'DISTANCE', 'DIRECTION', 'INTERSECTION', 'WEATHER_1', 'WEATHER_2', 'STATE_HWY_IND', 'CALTRANS_COUNTY', 'CALTRANS_DISTRICT', 'STATE_ROUTE', 'ROUTE_SUFFIX', 'POSTMILE_PREFIX', 'POSTMILE', 'LOCATION_TYPE', 'RAMP_INTERSECTION', 'SIDE_OF_HWY', 'TOW_AWAY', 'COLLISION_SEVERITY', 'NUMBER_KILLED', 'NUMBER_INJURED', 'PARTY_COUNT', 'PRIMARY_COLL_FACTOR', 'PCF_CODE_OF_VIOL', 'PCF_VIOL_CATEGORY', 'PCF_VIOLATION', 'PCF_VIOL_SUBSECTION', 'HIT_AND_RUN', 'TYPE_OF_COLLISION', 'MVIW', 'PED_ACTION', 'ROAD_SURFACE', 'ROAD_COND_1', 'ROAD_COND_2', 'LIGHTING', 'CONTROL_DEVICE', 'CHP_ROAD_TYPE', 'PEDESTRIAN_ACCIDENT', 'BICYCLE_ACCIDENT', 'MOTORCYCLE_ACCIDENT', 'TRUCK_ACCIDENT'

In [4]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# using primary and secondary street to find coordinate, take 10 mintues to run
collisions = pd.read_csv('/path/to/your/collisions.csv')  # update as needed

# Keep relevant columns (assuming PRIMARY_RD, SECONDARY_RD are present)
columns_needed = ['CASE_ID', 'POINT_X', 'POINT_Y', 'PRIMARY_RD', 'SECONDARY_RD']
collisions = collisions[columns_needed]

# Filter rows with missing coordinates
missing_coords = collisions[collisions['POINT_X'].isna() | collisions['POINT_Y'].isna()].copy()
print(f"❌ Missing coordinate rows: {len(missing_coords)}")

# Initialize geocoder
geolocator = Nominatim(user_agent="sd_collision_geocoder")

# Function to geocode an intersection
def geocode_intersection(primary, secondary, city="San Diego, CA"):
    try:
        query = f"{primary} & {secondary}, {city}"
        location = geolocator.geocode(query, timeout=10)
        if location:
            return pd.Series([location.longitude, location.latitude])
    except GeocoderTimedOut:
        time.sleep(1)
        return geocode_intersection(primary, secondary)  # retry
    return pd.Series([None, None])

# Apply geocoding
missing_coords[['POINT_X', 'POINT_Y']] = missing_coords.apply(
    lambda row: geocode_intersection(row['PRIMARY_RD'], row['SECONDARY_RD']),
    axis=1
)

# Drop rows still missing coordinates
filled_coords = missing_coords.dropna(subset=['POINT_X', 'POINT_Y'])

# Save to CSV
output_path = "/content/drive/MyDrive/traffic data/geocoded_missing_coords.csv"
filled_coords.to_csv(output_path, index=False)
print(f"✅ Geocoded results saved to: {output_path}")


❌ Missing coordinate rows: 593


KeyboardInterrupt: 

In [5]:
#Find still missing coordinate Total originally missing
total_missing = len(missing_coords)

# Rows that were successfully geocoded (POINT_X and POINT_Y filled)
total_filled = len(filled_coords)

# Remaining rows that failed to geocode
failed_geocode = total_missing - total_filled

print(f"🧮 Total rows originally missing coordinates: {total_missing}")
print(f"✅ Rows successfully geocoded: {total_filled}")
print(f"❌ Rows still missing coordinates after geocoding: {failed_geocode}")


NameError: name 'filled_coords' is not defined

In [48]:
# Count rows where DIRECTION is blank ("") or NaN
blank_direction = collisions['DIRECTION'].isna().sum() + (collisions['DIRECTION'].astype(str).str.strip() == '').sum()

print(f"❌ Number of rows with blank or missing DIRECTION: {blank_direction}")


❌ Number of rows with blank or missing DIRECTION: 4893


In [6]:
import pandas as pd

#show row where x and y are nan Keep only the selected columns
collisions = collisions[['CASE_ID', 'POINT_X', 'POINT_Y']]

# Count rows with missing coordinates
nan_count = collisions['POINT_X'].isna().sum() + collisions['POINT_Y'].isna().sum()
rows_with_nan = collisions[collisions['POINT_X'].isna() | collisions['POINT_Y'].isna()]

# Show the number of rows with NaNs
print(f"❌ Rows with missing POINT_X or POINT_Y: {len(rows_with_nan)}")
print("🔍 Sample rows with NaN coordinates:")
print(rows_with_nan.head())

# Optional: Show total rows
print(f"📊 Total rows in collisions: {len(collisions)}")




❌ Rows with missing POINT_X or POINT_Y: 593
🔍 Sample rows with NaN coordinates:
     CASE_ID  POINT_X  POINT_Y
23   5906018      NaN      NaN
42   5910590      NaN      NaN
77   5924114      NaN      NaN
134  5987410      NaN      NaN
262  6028830      NaN      NaN
📊 Total rows in collisions: 12833


In [7]:
# Drop rows where either POINT_X or POINT_Y is NaN
collisions = collisions.dropna(subset=['POINT_X', 'POINT_Y'])

# Confirm how many rows remain
print(f"✅ Remaining rows in 'collisions': {len(collisions)}")


✅ Remaining rows in 'collisions': 12240


In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


# Step 2: Convert to GeoDataFrame
gdf_points = gpd.GeoDataFrame(
    collisions,
    geometry=gpd.points_from_xy(collisions['POINT_X'], collisions['POINT_Y']),
    crs='EPSG:4326'
)

# Step 3: Load Census Tracts shapefile
tracts = gpd.read_file('/content/drive/MyDrive/traffic data/tl_2022_06_tract.shp')  # San Diego County tracts

# Step 4: Spatial join — assign tract to each point
collisions_with_tract = gpd.sjoin(gdf_points, tracts[['GEOID', 'NAMELSAD', 'geometry']], how='left', predicate='within')

# Step 5: View results
print(collisions_with_tract[['CASE_ID', 'POINT_X', 'POINT_Y', 'GEOID', 'NAMELSAD']].head())


   CASE_ID     POINT_X    POINT_Y        GEOID             NAMELSAD
0  5639604 -117.093777  32.694394  06073003305   Census Tract 33.05
1  5639619 -116.864564  32.853113  06073015501  Census Tract 155.01
2  5639726 -117.079670  32.735391  06073002705   Census Tract 27.05
3  5639817 -117.122654  32.917255  06073008355   Census Tract 83.55
4  5639818 -117.159290  32.990074  06073017061  Census Tract 170.61


<ipython-input-8-be1cb7a678c2>:17: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  collisions_with_tract = gpd.sjoin(gdf_points, tracts[['GEOID', 'NAMELSAD', 'geometry']], how='left', predicate='within')


In [9]:
# Count unmatched rows (no GEOID assigned)
unmatched_count = collisions_with_tract['GEOID'].isna().sum()

print(f"❌ Unmatched rows (no census tract found): {unmatched_count}")

# Optional: Show total and matched counts
total_rows = len(collisions_with_tract)
matched_count = total_rows - unmatched_count

print(f"✅ Matched rows: {matched_count}")
print(f"📊 Total rows: {total_rows}")


❌ Unmatched rows (no census tract found): 2
✅ Matched rows: 12238
📊 Total rows: 12240


In [11]:
# Step 1: Find unmatched rows (GEOID is NaN)
unmatched_rows = collisions_with_tract[collisions_with_tract['GEOID'].isna()]
unmatched_count = len(unmatched_rows)

# Step 2: Display unmatched rows
print(f"❌ Unmatched rows (no census tract found): {unmatched_count}")
print("🔍 Sample unmatched rows:")
print(unmatched_rows.head())

# Step 3: Remove unmatched rows
collisions_with_tract = collisions_with_tract[collisions_with_tract['GEOID'].notna()]

# Step 4: Show summary
matched_count = len(collisions_with_tract)
print(f"\n✅ Matched rows retained: {matched_count}")


❌ Unmatched rows (no census tract found): 0
🔍 Sample unmatched rows:
Empty GeoDataFrame
Columns: [CASE_ID, POINT_X, POINT_Y, geometry, index_right, GEOID, NAMELSAD]
Index: []

✅ Matched rows retained: 12238


In [12]:
collisions_with_tract

,CASE_ID,POINT_X,POINT_Y,geometry,index_right,GEOID,NAMELSAD
0,5639604,-117.093777,32.694394,POINT (-117.09378 32.69439),8668.0,06073003305,Census Tract 33.05
1,5639619,-116.864564,32.853113,POINT (-116.86456 32.85311),6126.0,06073015501,Census Tract 155.01
2,5639726,-117.079670,32.735391,POINT (-117.07967 32.73539),6021.0,06073002705,Census Tract 27.05
3,5639817,-117.122654,32.917255,POINT (-117.12265 32.91725),1169.0,06073008355,Census Tract 83.55
4,5639818,-117.159290,32.990074,POINT (-117.15929 32.99007),2846.0,06073017061,Census Tract 170.61
...,...,...,...,...,...,...,...
12827,9785219,-117.252281,33.178360,POINT (-117.25228 33.17836),6122.0,06073019702,Census Tract 197.02
12828,9785348,-117.234169,33.197689,POINT (-117.23417 33.19769),5954.0,06073019602,Census Tract 196.02
12829,9803657,-117.248878,32.934086,POINT (-117.24888 32.93409),6165.0,06073008324,Census Tract 83.24
12830,9816357,-117.031143,32.756508,POINT (-117.03114 32.75651),5868.0,06073014601,Census Tract 146.01


In [13]:
# Extract the numeric part from NAMELSAD and convert to float
collisions_with_tract['census_tract'] = (
    collisions_with_tract['NAMELSAD']
    .str.extract(r'(\d+\.\d+|\d+)$')[0]  # get the matched value from extraction
    .astype(float)  # convert to float
)

# Confirm result
print("✅ Extracted and converted 'census_tract':")
print(collisions_with_tract[['NAMELSAD', 'census_tract']].head())



✅ Extracted and converted 'census_tract':
              NAMELSAD  census_tract
0   Census Tract 33.05         33.05
1  Census Tract 155.01        155.01
2   Census Tract 27.05         27.05
3   Census Tract 83.55         83.55
4  Census Tract 170.61        170.61


In [14]:
print("📋 Data type of 'census_tract':", collisions_with_tract['census_tract'].dtype)


📋 Data type of 'census_tract': float64


In [15]:
rows_with_nan = collisions_with_tract.isna().any(axis=1).sum()
print(f"❌ Total rows with at least one NaN: {rows_with_nan}")


❌ Total rows with at least one NaN: 0


In [16]:
# Save to CSV
output_path = "/content/drive/MyDrive/traffic data/collisions_with_census_tract.csv"
collisions_with_tract.to_csv(output_path, index=False)

print(f"✅ Data saved to: {output_path}")


✅ Data saved to: /content/drive/MyDrive/traffic data/collisions_with_census_tract.csv


In [8]:
import pandas as pd

#Convert excel to csv Load Excel, skipping the first (empty) row and using second row as header
file_path = "/content/drive/MyDrive/traffic data/Client-Count-by-2024-Census-Tract-and-Cities.xlsx"
df = pd.read_excel(file_path, skiprows=1)

# Clean column names
df.columns = df.columns.str.strip()

# ✅ Rename the column
df = df.rename(columns={'2020 Census Tract': 'census_tract'})

# Drop rows where census_tract is missing
df = df.dropna(subset=['census_tract'])

# Save to CSV
output_path = "/content/drive/MyDrive/traffic data/2024_client_count_census.csv"
df.to_csv(output_path, index=False)

print(f"✅ Cleaned data saved to: {output_path}")







✅ Cleaned data saved to: /content/drive/MyDrive/traffic data/2024_client_count_census.csv


In [12]:
import pandas as pd

#Merge homeless count Load both datasets
collisions = pd.read_csv("/content/drive/MyDrive/traffic data/collisions_with_census_tract.csv")
client_count = pd.read_csv("/content/drive/MyDrive/traffic data/2024_client_count_census.csv")

# Clean and ensure consistent column format
collisions['census_tract'] = pd.to_numeric(collisions['census_tract'], errors='coerce').map(lambda x: f"{x:.2f}" if pd.notnull(x) else None)
client_count['census_tract'] = pd.to_numeric(client_count['census_tract'], errors='coerce').map(lambda x: f"{x:.2f}" if pd.notnull(x) else None)

# Merge on census_tract
merged = pd.merge(
    collisions,
    client_count,
    on='census_tract',
    how='left',
    suffixes=('', '_client')
)

# Find unmatched rows (i.e., no client count info)
unmatched = merged[merged.isnull().any(axis=1)]
num_unmatched = len(unmatched)

# Print results
print(f"❌ Number of unmatched collision records: {num_unmatched}")
print(f"📊 Total records in collisions: {len(collisions)}")
print(f"✅ Total matched: {len(merged) - num_unmatched}")

# Optional: Preview unmatched rows
print("\n🔍 Sample of unmatched collision rows:")
print(unmatched[['CASE_ID', 'census_tract']].head())



❌ Number of unmatched collision records: 2340
📊 Total records in collisions: 12238
✅ Total matched: 10155

🔍 Sample of unmatched collision rows:
    CASE_ID census_tract
1   5639619       155.01
4   5639818       170.61
8   5639956       191.09
10  5890297       213.06
16  5903367        83.39


In [13]:
#How many tract not matached Get unique unmatched census tracts from collision records
unmatched_tracts = unmatched['census_tract'].dropna().unique()

# Sort them for easy reading
unmatched_tracts_sorted = sorted(unmatched_tracts)

# Print results
print(f"🧾 Unique unmatched census tracts: {len(unmatched_tracts_sorted)}")
print("📋 List of unmatched census tracts:")
print(unmatched_tracts_sorted)


🧾 Unique unmatched census tracts: 304
📋 List of unmatched census tracts:
['1.00', '100.04', '100.05', '100.16', '100.17', '100.18', '101.04', '101.11', '104.01', '106.01', '108.00', '109.00', '11.00', '110.00', '111.00', '113.00', '118.01', '124.00', '133.01', '133.09', '133.15', '133.16', '133.18', '133.19', '133.21', '133.23', '133.24', '133.25', '133.26', '133.27', '134.01', '134.11', '134.12', '134.14', '134.17', '134.18', '134.22', '134.24', '134.25', '136.05', '136.07', '136.08', '139.05', '139.06', '141.01', '142.00', '146.01', '148.00', '152.00', '153.01', '153.02', '154.03', '154.05', '154.07', '155.01', '155.02', '156.02', '16.01', '161.00', '162.01', '166.08', '166.09', '166.10', '166.18', '167.03', '167.04', '167.06', '168.06', '168.07', '168.10', '168.11', '168.13', '169.02', '170.06', '170.09', '170.10', '170.14', '170.21', '170.31', '170.33', '170.34', '170.35', '170.36', '170.37', '170.39', '170.40', '170.41', '170.43', '170.44', '170.45', '170.46', '170.47', '170.48', 

In [14]:
#save to cvs Step 1: Filter out unmatched rows (keep only matched)
matched = merged[merged['Client Count'].notna()]

# Step 2: Save to CSV
output_path = "/content/drive/MyDrive/traffic data/collisions_with_client_count.csv"
matched.to_csv(output_path, index=False)

print(f"✅ Matched collisions with client count saved to: {output_path}")
print(f"📊 Rows saved: {len(matched)}")


✅ Matched collisions with client count saved to: /content/drive/MyDrive/traffic data/collisions_with_client_count.csv
📊 Rows saved: 10155
